#### 2. The data iterator performance is important. Do you think the current implementation is fast enough? Explore various options to improve it. Use a system profiler to find out where the bottlenecks are.

We can invoke the cProfile profiler as follows:

```python
with cProfile.Profile() as pr:
    for _ in data_loader:
        pass
    pr.print_stats(sort="tottime")
```

It prints the following results, indicating that the `poll` takes the majority of the time, followed by the `_share_filename_cpu_` function.

Explanation of what `poll` does by https://stackoverflow.com/a/50634084:
- Methods that execute inside a different process (for example with a ProcessPoolExecutor) are not captured by cProfile. So the select.poll is just showing your main process waiting for results from the other processes.

```python
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      942    2.029    0.002    2.029    0.002 {method 'poll' of 'select.poll' objects}
        4    0.137    0.034    0.137    0.034 {method '_share_filename_cpu_' of 'torch._C.StorageBase' objects}
      938    0.019    0.000    0.019    0.000 {built-in method _new_shared_filename_cpu}
        5    0.010    0.002    0.010    0.002 {built-in method _posixsubprocess.fork_exec}
      938    0.007    0.000    0.043    0.000 {built-in method _pickle.loads}

Given that the poll method is related to waiting for other threads it might make sense (especially considering our dataset is relatively small) to lower the number of workers that we use. This assumption turns out to be right as can be seen in the following profile (only goes up to 8 cpu cores as that is the maximum that PyTorch recommends for my system).

```python
Num Workers:    1 has a runtime of 1.58 ± 0.04 seconds
Num Workers:    2 has a runtime of 1.77 ± 0.01 seconds
Num Workers:    4 has a runtime of 2.18 ± 0.05 seconds
Num Workers:    8 has a runtime of 3.38 ± 0.16 seconds
```